In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy import table 
from astropy.io import ascii
import astropy.io.fits as pyfits
import os
import multiprocessing
from keplersplinev2 import *
import math
import threading
from scipy.stats import norm
from scipy.integrate import quad
from sympy import *
import time

In [209]:
#Get data
lctime = pd.read_csv("../18337/time_test.csv").to_numpy()
flux = pd.read_csv("../18337/flux_test.csv").to_numpy()
e_flux = pd.read_csv("../18337/e_flux_test.csv").to_numpy()
t0 = pd.read_csv("../18337/t0_test.csv").to_numpy()[0]
pdepths = pd.read_csv("../18337/pdepth_test.csv").to_numpy()[0]
P = pd.read_csv("../18337/P_test.csv").to_numpy()[0]

In [210]:
#Remove zeros from data arrays
def remove_zeros(A):
    B = [i for i in A if i != 0]
    return B

In [159]:
#MCMC
def log_likelihood(y, x, params, sigma, model, P):
    #Log-likelihood function
    y_pred = model(x, params, P)  #Predicted value using current params
    ll = np.sum(norm.logpdf(y_pred, loc=y, scale=sigma))
    return ll

def sampler(y, x, p0, priors, sigma, model, n_iterations, chain, log_prior, P):
    #Define sampler
    n_params = len(p0)
    lp = log_likelihood(y, x, p0, sigma, model, P)
    lp += log_prior(*p0, priors)
    for i in range(1, n_iterations):
        p_new = chain[:, i-1] + np.random.normal(0, 0.1, size=n_params)
        lp_new = log_likelihood(y, x, p_new, sigma, model, P) + log_prior(*p_new, priors)
        if lp_new - lp > np.log(np.random.rand()):
            chain[:, i] = p_new
            lp = lp_new
        else:
            chain[:, i] = chain[:, i-1]
    return chain

In [131]:
#using mandel & agol 2002
def log_prior_MA(t0, gam1, gam2, p, Ra, priors):
    #Define priors
    t0_prior, gam1_prior, gam2_prior, p_prior, Ra_prior = priors
    lp = norm.logpdf(t0_prior, t0) + norm.logpdf(gam1_prior, gam1) + norm.logpdf(gam2_prior, gam2) + norm.logpdf(p_prior, p) + norm.logpdf(Ra_prior, Ra)
    return lp

def log_prior_simple(t0, depth, duration, priors):
    #Define priors
    t0_prior, depth_prior, duration_prior = priors
    lp = norm.logpdf(t0_prior, t0) + norm.logpdf(depth_prior, depth) + norm.logpdf(duration_prior, duration) 
    return lp

def find_I(r,gam1,gam2):
    mu = np.sqrt(1-r**2)
    return 1 - gam1*(1-mu) - gam2*(1-mu)**2
    
def find_F(p,z):
    if 1+p<z: 
        lamb = 0
    elif np.abs(1-p)<z and z<=(1+p):
        coeff1 = p**2 * np.arccos((p**2+z**2-1)/(2*p*z))
        coeff2 = np.arccos((1-p**2+z**2)/(2*z))
        coeff3 = np.sqrt((4*z**2 - (1+z**2-p**2)**2)/4)
        lamb = (1/np.pi)*(coeff1+coeff2-coeff3)
    elif z<=(1-p):
        lamb = p**2
    else:
        lamb = 1
    return 1 - lamb
    
def find_dFdr(p,z,r):
    p2 = p/r #Used for checking conditions
    z2 = z/r 
    if 1+p2<z2: 
        lamb = 0
    elif np.abs(1-p2)<z2 and z2<(1+p2):
        #Coeff1
        t1 = p/(r*z*np.sqrt(1-((p**2-r**2+z**2)**2/(4*p**2*z**2))))
        k0 = np.arccos((p**2+z**2-r**2)/(2*p*z))
        t2 = 2*p**2*k0/r**3
        coeff1 = t1 - t2
        #Coeff2
        num = p**2/r**2 - z**2/r**2 + 1
        denom = 2*z*np.sqrt(1-((-p**2/r+z**2/r+r)**2/(4*z**2)))
        coeff2 = num/denom
        #Coeff3
        num = 2*(2*p**2/r**3-2*z**2/r**3)*(-p**2/r**2+z**2/r**2+1)+(8*z**2/r**3)
        denom = 4*np.sqrt(4*z**2/r**2-(-p**2/r**2+z**2/r**2+1)**2)
        coeff3 = num/denom
        #Combine
        lamb = (1/np.pi)*(coeff1-coeff2+coeff3)
    elif z2<(1-p2):
        lamb = -2*p**2 / r**3
    else:
        lamb = 0
    return -1*lamb
    
def find_dFdr_r(p,z,r):
    return find_F(p/r,z/r)*2*r + find_dFdr(p,z,r)*r**2 
   
def integrand1(r,gam1,gam2):
    return 2*r*find_I(r,gam1,gam2)

def integrand2(r,gam1,gam2,p,z):
    return find_dFdr_r(p,z,r)*find_I(r,gam1,gam2)

def Mandel_Agol_model(x, p, P): 
    #Define Mandel Agol model
    t0, gam1, gam2, p, Ra = p
    w = 2*np.pi/P
    all_F = []
    for i in range(len(x)):
        z = (1/Ra)*(np.sin(w*(x[i]-t0))**2)**.5
        I1 = quad(integrand1, 0, 1, args=(gam1,gam2))[0]
        I2 = quad(integrand2, 0, 1, args=(gam1,gam2,p,z))[0]
        F = I2/I1 
        all_F.append(F)
    return all_F

def simple_model(x, p, P):
    # Define Box model
    t0, depth, duration = p
    box = np.ones(x.shape)
    box[(x >= t0) & (x <= t0 + duration)] = 1 - depth
    return box

In [273]:
#Now fit model 
def fitter(i, parallel, model):
    #Remove zeros from lightcurve
    x = np.asarray(remove_zeros(lctime[:,i]))
    y = remove_zeros(flux[:,i])
    e_y = remove_zeros(e_flux[:,i])

    #Initial guesses -- made smarter with Kepler results
    #params: t0, gam1, gam2, p (Rp/Rs), Ra (Rs/semimajor axis)
    if model=='simple':
        t0_guess = t0[i]
        p0 = [t0[i]-.04, 0.05, .08]
        perts = [.1,.01,.02] #perturbations
        #Priors
        t0_prior = np.random.uniform(t0[i]-.2, t0[i]+.2) 
        depth_prior = np.random.uniform(0.01, .1) 
        duration_prior = np.random.uniform(0.01, .5) 
        priors = [t0_prior, depth_prior, duration_prior]

    elif model=='MA':
        t0_guess = t0[i]
        p_guess = pdepths[i]**.5
        p0 = [t0[i],.18,.15,pdepths[i]**.5,.05]
        perts = [.01,.02,.02,.02,.1] #perturbations
        #Priors
        t0_prior = np.random.uniform(t0[i]-.2, t0[i]+.2) 
        gam1_prior = np.random.uniform(-1, 1)
        gam2_prior = np.random.uniform(-1, 1)
        p_prior = np.random.uniform(0, .5)
        Ra_prior = np.random.uniform(.001, 1)
        priors = [t0_prior, gam1_prior, gam2_prior, p_prior, Ra_prior]
        
    #Sampler parameters
    n_iters = 100 #10000
    burnin = 8 #2000
    nchains = 128 #128

    #Initialize chain
    chains = np.zeros((len(p0), n_iters, nchains))
    params_init = [p0 + np.random.randn(len(p0))*perts for i in range(nchains)]
      
    call_model = simple_model if model=='simple' else Mandel_Agol_model
    log_prior = log_prior_simple if model=='simple' else log_prior_MA
    #Loop through chains -- parallel or serial
    if parallel:
        for thd in range(4):
            for chain_ct in range(nchains):
                if chain_ct%4==thd:
                    chains[:, 0,chain_ct] = np.asarray(params_init[chain_ct])
                    chains[:,:,chain_ct] = sampler(y, x, p0, priors, e_y, call_model, n_iters, chains[:,:,chain_ct], log_prior, P[i])

    else:
        for chain_ct in range(nchains):
            chains[:, 0,chain_ct] = params_init[chain_ct]
            chains[:,:,chain_ct] = sampler(y, x, p0, priors, e_y, call_model, n_iters, chains[:,:,chain_ct], log_prior, P[i])

    #Calculate mean of each parameter
    if model == 'simple':
        t0_mean = np.mean(chains[0, burnin:-1,:])
        depth_mean = np.mean(chains[1, burnin:-1,:])
        duration_mean = np.mean(chains[2, burnin:-1,:])
        fitparams = [t0_mean, depth_mean, duration_mean]
    elif model=='MA':
        t0_mean = np.mean(chains[0, burnin:-1,:])
        gam1_mean = np.mean(chains[1, burnin:-1,:])
        gam2_mean = np.mean(chains[2, burnin:-1,:])
        p_mean = np.mean(chains[2, burnin:-1,:])
        Ra_mean = np.mean(chains[2, burnin:-1,:])
        fitparams = [t0_mean, gam1_mean, gam2_mean, p_mean, Ra_mean]

    #Calculate goodness of fit using RMSE
    prediction = call_model(x, fitparams, P[i]) 
    rmse = sqrt(sum((y-prediction)**2)/len(y)) 

    return fitparams, rmse


def iterate(ndata, parallel):
    #Iterate through data
    all_rmse = zeros(ndata)
    for i in range(ndata):
        fitparams_i, rmse_i = fit_simple(i, parallel)
        all_rmse[i] = rmse_i
    return all_rmse

In [153]:
#Example fitting call using simple model, serial implementation
start = time.time()
params, rmse = fitter(0,False,model='simple')
end = time.time()
print(end - start)

52.069961071014404
